<a href="https://colab.research.google.com/github/mr-alamdari/Multi-Object-Segmentation-Pytorch/blob/main/Multi_Object_Segmentation_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import copy
import torch
import torchvision
import torchsummary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#1.Data

####Visual Object Classes(VOC) DataSet